In [ ]:
# We need to pip install these two versions because it gives me some errors otherwise

# This can take some time

!pip install "torch==1.4" "torchvision==0.5.0"

     |████████████████████████████████| 753.4MB 19kB/s 
     |████████████████████████████████| 4.0MB 36.3MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/MyDrive/Mango/mangoimg/smallplant"

 22309112_112df74058_z.jpg
 53bf64ddac86d20cdaf21296bc92ecd5.jpg
 ac42c5bed35e2c8f115508b4d1434510.jpg
'Avocado, Avocadobaum züchten – Pflanzen und Pflege - Outdoor Ideas.jpg'
'How To Grow A Mango Tree From Seed.jpg'
'How to Grow a Mango Tree.jpg'
'How to Grow Mango from Seed (Easy Method) _ Empress of Dirt.jpg'
'How to Grow Mango from Seed (Easy Method) _ Empress of.jpg'
 IMG_4839.JPG
'kesar-mango-tree-plants-1996573 (1).jpg'
 kesar-mango-tree-plants-1996573.jpg
 LEAF-7106-1.jpg
 manggo02_600x712.png
'Mangobaum selber ziehen.jpg'
 mango-pit.jpg
 mango-plant-1501571897-3184199.jpeg
 mango-plant-500x500.jpg
 mango-plant-green-leaves-flowerpot-white-background-137784517.jpg
 mango-plant-green-leaves-flowerpot-white-background-138318881.jpg
 mango-pot.jpg
 mango-r2e2-seedling-5857.jpeg
 mango-seedling.jpg
'Mango tree bonsai.jpg'
 mango-tree-in-pot.jpg
'mango tree.jpg'
'Mango Tree Pruning _ How To Train A Mango Tree.jpg'
 mango-tree-seedling.jpg
 mango-tree-small-mango-tree-mango-seedling

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle datasets download -d alexo98/leaf-detection

leaf-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
from PIL import Image
import time
!pip install albumentations==0.4.6 
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.pytorch import transforms

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt


DIR_INPUT = '/content/gdrive/MyDrive/Kaggle'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

# Loading the device now
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


     |████████████████████████████████| 122kB 9.5MB/s 
     |████████████████████████████████| 952kB 14.9MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=65164 sha256=42afd38554b62f6334109883f80b4b71c86230f8f77fa5d035bc4aa897f6d0bd
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
#!unzip -q "/content/gdrive/MyDrive/Kaggle/leaf-detection.zip"

In [ ]:
train_df = pd.read_csv(os.path.join(DIR_INPUT,"train.csv"))

In [ ]:
class LeafDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)                
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)
        image = np.reshape(image,image.shape+(1,))
        image /= 255.0

        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]


In [ ]:
# This Albumentation for now it is empty.
def transform():
    return A.Compose([        
#         A.Cutout(num_holes=10,max_h_size=15,max_w_size=15,p=1),
            
#         A.OneOf([
#             A.RandomSunFlare(src_radius=200,num_flare_circles_lower=6,num_flare_circles_upper=8,p=1),
#             A.RandomRain(slant_lower=-10,slant_upper=10,drop_length=20,drop_width=1,p=1),
#             A.RandomFog(fog_coef_lower=0.05, fog_coef_upper=0.1, alpha_coef=0.08, p=1),  
#         ], p=1),
#         A.OneOf([
#             A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=True, p=1),
#             A.RandomGamma(gamma_limit=(80,165),p=1),  
#         ], p=1),      
        ToTensorV2(p=1.0),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def collate_fn(batch):
    return tuple(zip(*batch))


In [ ]:
train_dataset = LeafDataset(train_df, DIR_TRAIN, transform())
valid_dataset = LeafDataset(valid_df, DIR_TRAIN, transform())


train_data_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

NameError: ignored

In [ ]:
!pip install albumentations==0.4.6
from albumentations.pytorch.transforms import ToTensorV2
# This Albumentation for now it is empty.
def transform():
    return A.Compose([        
#         A.Cutout(num_holes=10,max_h_size=15,max_w_size=15,p=1),
            
#         A.OneOf([
#             A.RandomSunFlare(src_radius=200,num_flare_circles_lower=6,num_flare_circles_upper=8,p=1),
#             A.RandomRain(slant_lower=-10,slant_upper=10,drop_length=20,drop_width=1,p=1),
#             A.RandomFog(fog_coef_lower=0.05, fog_coef_upper=0.1, alpha_coef=0.08, p=1),  
#         ], p=1),
#         A.OneOf([
#             A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=True, p=1),
#             A.RandomGamma(gamma_limit=(80,165),p=1),  
#         ], p=1),      
        ToTensorV2(p=1.0),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def collate_fn(batch):
    return tuple(zip(*batch))